In [1]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

import sys
backtest_dir = 'C://backtest/backtest/'
if backtest_dir not in sys.path:
    sys.path.insert(0, backtest_dir)
    
from Backtest import *
from BayesianOptimization import *
from MACD_EMVStrategy import MACD_EMVStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [2]:
def run_backtest(config, trading_data, ohlc_data, short_window, delta_window, window, n, m):
    short_window = int(short_window)
    long_window = short_window + int(delta_window)
    window = int(window)
    n = int(n)
    m = int(m)
    config['title'] = "MACD_EMVStrategy" + "_" + str(short_window) + "_" + str(long_window) + "_" + str(window) + "_" + str(n) + "_" + str(m)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")

    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data = trading_data, ohlc_data = ohlc_data
    )
    strategy = MACD_EMVStrategy(config, events_queue, data_handler,
                            short_window=short_window, long_window=long_window,
                           window=window, n=n, m=m)

    backtest = Backtest(config, events_queue, strategy,
                        data_handler= data_handler)

    results = backtest.start_trading()
    
    return (results['cum_returns'][-1] - 1)

## Setting

In [3]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/MACD_EMVStrategy",
    "title": "MACD_EMVStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2017-07-01T00:0:00", freq = "60" + "T"),    # str(freq) + "T"
    "end_date": pd.Timestamp("2018-04-01T00:00:00", freq = "60" + "T"),
    "equity": 1.0,
    "freq": 60,      # min
    "commission_ratio": 0.001,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT']
}

In [4]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [5]:
gp_params = {"alpha": 1e-5}

## Acquisition Function "Upper Confidence Bound"

* Prefer exploitation (kappa=1.0)

* Prefer exploration (kappa=10)


## Acquisition Function "Expected Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [6]:
BO = BayesianOptimization(
    run_backtest,
    {'short_window': (1, 120),
    'delta_window': (1, 120),
    'window': (1, 120),
     'n': (1, 120), 
     'm': (1, 120)},
    is_int = [1, 1, 1, 1, 1], 
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
# BO.explore({
#     'short_window': np.arange(1, 120, 20),
#     'delta_window': np.arange(1, 120, 20)
#     },
#     eager=True)
BO.maximize(init_points=80, n_iter=120, acq="ei", xi=0.01, **gp_params)

Initialization
---------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   delta_window |         m |         n |   short_window |    window | 
---------------------------------
MACD_EMVStrategy_38_126_61_17_114
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.4010422183
Sortino Ratio: 3.0583889738
Max Drawdown: 0.3671448388
Max Drawdown Duration: 273500
Total Returns: 0.8876032580
Annualized Returns: 2.0890393714
Compound Annual Growth Rate: 3.4604236807
---------------------------------
Trades: 16
Trade Winning: 68.75%
Average Trade: 4.83%
Average Win: 10.06%
Average Loss: -6.69%
Best Trade: 27.05%
Worst Trade: -15.35%
Worst Trade Date: 2 days 

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.1807521330
Sortino Ratio: 3.1373332257
Max Drawdown: 0.2392715757
Max Drawdown Duration: 250900
Total Returns: 0.7457696354
Annualized Returns: 1.7552235373
Compound Annual Growth Rate: 2.7113678533
---------------------------------
Trades: 65
Trade Winning: 53.85%
Average Trade: 1.23%
Average Win: 4.66%
Average Loss: -2.76%
Best Trade: 31.90%
Worst Trade: -12.04%
Worst Trade Date: 0 days 17:00:00
Avg Days in Trade: 0 days 18:13:51
---------------------------------
   10 | 00m26s |    0.74577 | 24.000000 | 18.000000 | 1.000000 | 17.000000 | 104.000000 | 
---------------------------------
MACD_EMVStrategy_2_103_99_30_69
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complet

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.1922391683
Sortino Ratio: 1.3668013383
Max Drawdown: 0.4968202504
Max Drawdown Duration: 250900
Total Returns: 0.3144367877
Annualized Returns: 0.7400500429
Compound Annual Growth Rate: 0.9031043672
---------------------------------
Trades: 16
Trade Winning: 56.25%
Average Trade: 2.75%
Average Win: 11.33%
Average Loss: -8.29%
Best Trade: 33.39%
Worst Trade: -21.41%
Worst Trade Date: 3 days 19:00:00
Avg Days in Trade: 3 days 18:30:00
---------------------------------
   19 | 00m21s |    0.31444 | 35.000000 | 82.000000 | 85.000000 | 102.000000 | 67.000000 | 
---------------------------------
MACD_EMVStrategy_19_30_72_84_29
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest compl

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.3048952124
Sortino Ratio: 1.4523930644
Max Drawdown: 0.3863009181
Max Drawdown Duration: 273500
Total Returns: 0.2987852418
Annualized Returns: 0.7032129818
Compound Annual Growth Rate: 0.8501991168
---------------------------------
Trades: 24
Trade Winning: 45.83%
Average Trade: 1.67%
Average Win: 9.11%
Average Loss: -4.62%
Best Trade: 21.56%
Worst Trade: -16.70%
Worst Trade Date: 0 days 22:00:00
Avg Days in Trade: 1 days 17:25:00
---------------------------------
   28 | 00m13s |    0.29879 | 93.000000 | 103.000000 | 29.000000 | 69.000000 | 21.000000 | 
---------------------------------
MACD_EMVStrategy_88_163_119_95_113
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest com

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.4083980655
Sortino Ratio: 1.3354223955
Max Drawdown: 0.4178528246
Max Drawdown Duration: 250900
Total Returns: 0.3869999223
Annualized Returns: 0.9108327027
Compound Annual Growth Rate: 1.1596695989
---------------------------------
Trades: 12
Trade Winning: 58.33%
Average Trade: 3.74%
Average Win: 11.30%
Average Loss: -6.84%
Best Trade: 35.68%
Worst Trade: -16.46%
Worst Trade Date: 1 days 07:00:00
Avg Days in Trade: 3 days 17:20:00
---------------------------------
   37 | 00m22s |    0.38700 | 114.000000 | 87.000000 | 78.000000 | 106.000000 | 54.000000 | 
---------------------------------
MACD_EMVStrategy_10_88_85_51_17
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest comp

...

---------------------------------
MACD_EMVStrategy_1_29_27_105_35
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.2680534930
Sortino Ratio: 4.0080697801
Max Drawdown: 0.2249110161
Max Drawdown Duration: 168600
Total Returns: 0.8739474545
Annualized Returns: 2.0568994362
Compound Annual Growth Rate: 3.3848482548
---------------------------------
Trades: 85
Trade Winning: 29.41%
Average Trade: 1.09%
Average Win: 7.21%
Average Loss: -1.46%
Best Trade: 33.13%
Worst Trade: -4.30%
Worst Trade Date: 0 days 04:00:00
Avg Days in Trade: 0 days 13:58:35
---------------------------------
  163 | 00m36s |    0.87395 | 28.000000 | 35.000000 | 105.000000 | 1.000000 | 27.000000 | 
---------------------------------
MACD_EMVStrategy_

---------------------------------
MACD_EMVStrategy_120_240_120_51_120
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.4650090117
Sortino Ratio: 0.5972022674
Max Drawdown: 0.3694711499
Max Drawdown Duration: 273500
Total Returns: 0.0416412842
Annualized Returns: 0.0980058166
Compound Annual Growth Rate: 0.1007813080
---------------------------------
Trades: 11
Trade Winning: 54.55%
Average Trade: 1.38%
Average Win: 11.35%
Average Loss: -10.58%
Best Trade: 26.56%
Worst Trade: -14.95%
Worst Trade Date: 4 days 17:00:00
Avg Days in Trade: 4 days 20:49:05
---------------------------------
  172 | 01m19s |    0.04164 | 120.000000 | 120.000000 | 51.000000 | 120.000000 | 120.000000 | 
---------------------------------
MACD_E

---------------------------------
MACD_EMVStrategy_80_200_1_36_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  181 | 00m54s |    0.00000 | 120.000000 | 1.000000 | 36.000000 | 80.000000 | 1.000000 | 
---------------------------------
MACD_EMVStrategy_64_94_110_51_108
---------------------------------
Data Time Int

---------------------------------
MACD_EMVStrategy_110_130_76_99_110
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.0702549685
Sortino Ratio: -0.0834609305
Max Drawdown: 0.5769380689
Max Drawdown Duration: 250900
Total Returns: -0.1424870387
Annualized Returns: -0.3353536967
Compound Annual Growth Rate: -0.3035706355
---------------------------------
Trades: 19
Trade Winning: 42.11%
Average Trade: 0.04%
Average Win: 9.54%
Average Loss: -6.87%
Best Trade: 41.27%
Worst Trade: -15.84%
Worst Trade Date: 1 days 08:00:00
Avg Days in Trade: 3 days 06:03:09
---------------------------------
  190 | 00m57s |   -0.14249 | 20.000000 | 110.000000 | 99.000000 | 110.000000 | 76.000000 | 
---------------------------------
MACD_E

---------------------------------
MACD_EMVStrategy_39_88_75_17_8
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.9804529981
Sortino Ratio: 2.5593280595
Max Drawdown: 0.3367408033
Max Drawdown Duration: 273500
Total Returns: 0.6690035985
Annualized Returns: 1.5745490389
Compound Annual Growth Rate: 2.3386393582
---------------------------------
Trades: 29
Trade Winning: 55.17%
Average Trade: 2.30%
Average Win: 7.73%
Average Loss: -4.39%
Best Trade: 20.90%
Worst Trade: -10.84%
Worst Trade Date: 1 days 00:00:00
Avg Days in Trade: 1 days 20:49:39
---------------------------------
  199 | 00m45s |    0.66900 | 49.000000 | 8.000000 | 17.000000 | 39.000000 | 75.000000 | 
---------------------------------
MACD_EMVStrategy_2

In [7]:
print(BO.res['max'])

{'max_val': 1.7327963407657383, 'max_params': {'short_window': 17.0, 'delta_window': 31.0, 'window': 23.0, 'n': 112.0, 'm': 43.0}}


In [8]:
BO.maximize(init_points=0, n_iter=120, acq="ei", xi=0.0001, **gp_params)

Bayesian Optimization
---------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   delta_window |         m |         n |   short_window |    window | 
---------------------------------
MACD_EMVStrategy_45_129_1_78_70
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.1739301636
Sortino Ratio: -1.3849490888
Max Drawdown: 0.6701658057
Max Drawdown Duration: 273500
Total Returns: -0.3899981610
Annualized Returns: -0.9178892773
Compound Annual Growth Rate: -0.6875639758
---------------------------------
Trades: 75
Trade Winning: 34.67%
Average Trade: -0.33%
Average Win: 3.89%
Average Loss: -2.58%
Best Trade: 21.95%
Worst Trade: -9.80%
Worst Trade Date

---------------------------------
MACD_EMVStrategy_84_156_1_73_97
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.3095964919
Sortino Ratio: 0.4179038013
Max Drawdown: 0.4734208976
Max Drawdown Duration: 273500
Total Returns: 0.0005124390
Annualized Returns: 0.0012060629
Compound Annual Growth Rate: 0.0012064812
---------------------------------
Trades: 62
Trade Winning: 40.32%
Average Trade: 0.33%
Average Win: 4.08%
Average Loss: -2.20%
Best Trade: 21.60%
Worst Trade: -13.94%
Worst Trade Date: 3 days 00:00:00
Avg Days in Trade: 0 days 14:59:02
---------------------------------
  210 | 00m39s |    0.00051 | 72.000000 | 97.000000 | 73.000000 | 84.000000 | 1.000000 | 
---------------------------------
MACD_EMVStrategy_

---------------------------------
MACD_EMVStrategy_72_114_113_49_87
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.5609575079
Sortino Ratio: -0.6548698798
Max Drawdown: 0.6018245315
Max Drawdown Duration: 273500
Total Returns: -0.2461392829
Annualized Returns: -0.5793068561
Compound Annual Growth Rate: -0.4857247374
---------------------------------
Trades: 15
Trade Winning: 46.67%
Average Trade: -1.10%
Average Win: 8.25%
Average Loss: -9.29%
Best Trade: 22.14%
Worst Trade: -17.94%
Worst Trade Date: 2 days 19:00:00
Avg Days in Trade: 3 days 19:56:00
---------------------------------
  219 | 00m51s |   -0.24614 | 42.000000 | 87.000000 | 49.000000 | 72.000000 | 113.000000 | 
---------------------------------
MACD_EM

---------------------------------
MACD_EMVStrategy_1_121_1_23_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  228 | 00m39s |    0.00000 | 120.000000 | 1.000000 | 23.000000 | 1.000000 | 1.000000 | 
---------------------------------
MACD_EMVStrategy_93_112_72_26_87
---------------------------------
Data Time Interv

---------------------------------
MACD_EMVStrategy_65_110_10_104_113
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.0778314036
Sortino Ratio: 2.6499080922
Max Drawdown: 0.3336422256
Max Drawdown Duration: 273500
Total Returns: 0.6748599464
Annualized Returns: 1.5883323834
Compound Annual Growth Rate: 2.3662767718
---------------------------------
Trades: 19
Trade Winning: 47.37%
Average Trade: 3.44%
Average Win: 12.17%
Average Loss: -4.42%
Best Trade: 26.86%
Worst Trade: -14.85%
Worst Trade Date: 2 days 16:00:00
Avg Days in Trade: 3 days 00:25:16
---------------------------------
  237 | 00m39s |    0.67486 | 45.000000 | 113.000000 | 104.000000 | 65.000000 | 10.000000 | 
---------------------------------
MACD_EMVSt

...

---------------------------------
MACD_EMVStrategy_9_67_16_9_6
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.9602070643
Sortino Ratio: 3.5782373056
Max Drawdown: 0.2590269879
Max Drawdown Duration: 203900
Total Returns: 0.6333493694
Annualized Returns: 1.4906341955
Compound Annual Growth Rate: 2.1731986821
---------------------------------
Trades: 77
Trade Winning: 50.65%
Average Trade: 0.97%
Average Win: 4.62%
Average Loss: -2.78%
Best Trade: 24.71%
Worst Trade: -10.20%
Worst Trade Date: 0 days 04:00:00
Avg Days in Trade: 0 days 14:23:23
---------------------------------
  291 | 00m55s |    0.63335 | 58.000000 | 6.000000 | 9.000000 | 9.000000 | 16.000000 | 
---------------------------------
MACD_EMVStrategy_1_121

---------------------------------
MACD_EMVStrategy_68_132_74_44_90
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.2646265074
Sortino Ratio: 0.3364954406
Max Drawdown: 0.5792695865
Max Drawdown Duration: 250900
Total Returns: -0.0384287425
Annualized Returns: -0.0904448642
Compound Annual Growth Rate: -0.0881032896
---------------------------------
Trades: 15
Trade Winning: 46.67%
Average Trade: 0.81%
Average Win: 12.23%
Average Loss: -9.18%
Best Trade: 29.69%
Worst Trade: -19.87%
Worst Trade Date: 2 days 12:00:00
Avg Days in Trade: 4 days 00:08:00
---------------------------------
  300 | 01m35s |   -0.03843 | 64.000000 | 90.000000 | 44.000000 | 68.000000 | 74.000000 | 
---------------------------------
MACD_EMVStr

---------------------------------
MACD_EMVStrategy_106_146_45_50_97
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.2825555333
Sortino Ratio: -0.2735172999
Max Drawdown: 0.5370900196
Max Drawdown Duration: 273500
Total Returns: -0.1661735845
Annualized Returns: -0.3911017195
Compound Annual Growth Rate: -0.3480027680
---------------------------------
Trades: 20
Trade Winning: 45.00%
Average Trade: -0.35%
Average Win: 7.53%
Average Loss: -6.79%
Best Trade: 15.79%
Worst Trade: -14.64%
Worst Trade Date: 0 days 18:00:00
Avg Days in Trade: 2 days 08:03:00
---------------------------------
  309 | 01m55s |   -0.16617 | 40.000000 | 97.000000 | 50.000000 | 106.000000 | 45.000000 | 
---------------------------------
MACD_EM

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.19161902e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 2, 'warnflag': 2}
  " state: %s" % convergence_dict)


---------------------------------
MACD_EMVStrategy_120_196_90_101_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  316 | 02m12s |    0.00000 | 76.000000 | 1.000000 | 101.000000 | 120.000000 | 90.000000 | 
---------------------------------
MACD_EMVStrategy_75_106_89_40_54
---------------------------------
Data Time

In [9]:
print(BO.res['max'])

{'max_val': 2.371979954138366, 'max_params': {'short_window': 16.0, 'delta_window': 29.0, 'window': 31.0, 'n': 113.0, 'm': 41.0}}


In [10]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

,Parameters,Target
134,"[1.0, 1.0, 120.0, 120.0, 120.0]",-0.871441
255,"[85.0, 1.0, 1.0, 1.0, 120.0]",-0.789482
147,"[1.0, 1.0, 120.0, 19.0, 55.0]",-0.784204
148,"[120.0, 120.0, 1.0, 1.0, 120.0]",-0.780283
157,"[1.0, 6.0, 1.0, 120.0, 103.0]",-0.631969
248,"[1.0, 5.0, 69.0, 71.0, 73.0]",-0.622273
276,"[11.0, 9.0, 102.0, 116.0, 97.0]",-0.584467
233,"[48.0, 73.0, 4.0, 13.0, 3.0]",-0.555013
200,"[45.0, 84.0, 1.0, 78.0, 70.0]",-0.389998
140,"[92.0, 21.0, 114.0, 101.0, 11.0]",-0.377633


## Acquisition Function "Probability of Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)